In [1]:
import json
import pickle
from typing import List

import deeppavlov
import pandas as pd
import torch
import tqdm

### 1) Загружаем RuBERT для эмбеддингов

In [2]:
from deeppavlov.core.common.file import read_json
from deeppavlov import build_model, configs

path = configs.embedder.bert_embedder
bert_config = read_json(path)
bert_config['metadata']['variables']['BERT_PATH'] = '{DOWNLOADS_PATH}/bert_models/rubert_cased_L-12_H-768_A-12_pt'
bert_config['metadata']['download'][0]['url'] = 'http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_pt.tar.gz'


m = build_model(bert_config, download=True)

2020-11-01 21:56:14.428 INFO in 'deeppavlov.download'['download'] at line 132: Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_pt.tar.gz download because of matching hashes
[nltk_data] Downloading package punkt to /home/mikhail/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mikhail/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/mikhail/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/mikhail/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!


In [3]:
texts = ['сегодня мы будем готовить офигенный омлет', 'второе предложение']
tokens, token_embs, subtokens, subtoken_embs, sent_max_embs, sent_mean_embs, bert_pooler_outputs = m(texts)
print(tokens)
print(token_embs)
print(sent_max_embs)

[['сегодня', 'мы', 'будем', 'готовить', 'офигенный', 'омлет'], ['второе', 'предложение']]
[array([[ 0.16702184, -0.49682602, -0.04930846, ...,  0.6940326 ,
        -0.35982978, -0.24980858],
       [ 0.23081881, -0.26429278,  0.26739708, ..., -0.01405226,
        -0.4066898 , -0.25012636],
       [ 0.52555764, -0.14646043, -0.28406313, ..., -0.03530244,
        -0.11871938, -0.46033123],
       [ 0.8812914 , -0.05890733,  0.13671885, ...,  0.56064916,
        -0.25506568,  0.11571673],
       [ 0.25530246, -0.15127186,  0.02684454, ...,  0.1628736 ,
         0.6039251 ,  0.15912056],
       [ 0.60406923, -0.09385679, -0.5199474 , ...,  0.5782663 ,
         0.03793397,  0.28421792]], dtype=float32), array([[ 0.42386067,  0.18435678,  0.712276  , ...,  0.06040264,
        -0.31401923, -0.23336643],
       [ 0.4937695 ,  0.3234263 ,  0.388501  , ..., -0.13395219,
         0.26636454, -0.18983997]], dtype=float32)]
[[0.95064837 0.06589679 0.57732314 ... 0.95861495 0.6039251  0.28421792]
 [

### 2) Загруэаем наш датасет

In [4]:
data = pd.read_csv('data/russianfood_filtred.csv')
data.head()

,id,url,title,ingredients,steps,images
0,158659,https://www.russianfood.com/recipes/recipe.php...,Запеканка из куриного филе с грибами и беконом...,Курица (любая часть) - 600 г|Бекон - 100 г|Сме...,Подготовьте все необходимые ингредиенты. Сыр м...,images/158659_0.jpg|images/158659_1.jpg|images...
1,146718,https://www.russianfood.com/recipes/recipe.php...,Сырные лепешки с картошкой (на сухой сковороде),Для теста:|Сыр (твердый или полутвердый) - 250...,"Как приготовить лепешки из сырного теста, с ка...",images/146718_0.jpg|images/146718_1.jpg|images...
2,134548,https://www.russianfood.com/recipes/recipe.php...,"Торт без выпечки, с творогом и черносливом",Печенье песочное (сахарное) – 600 г|Творог – 3...,"Как приготовить торт без выпечки, с творогом и...",images/134548_0.jpg|images/134548_1.jpg|images...
3,151796,https://www.russianfood.com/recipes/recipe.php...,Сырные булочки с помидорами и зелёным луком,Для теста:|Сахар - 1 ст. ложка|Яйцо - 1 шт.|ил...,Для теста будем использовать прессованные дрож...,images/151796_0.jpg|images/151796_1.jpg|images...
4,114586,https://www.russianfood.com/recipes/recipe.php...,Курица в сливках с черносливом,Куриные голени - 4 шт.|Лук репчатый (крупный) ...,Подготовьте все необходимые ингредиенты. Идеал...,images/114586_0.jpg|images/114586_1.jpg|images...


### 3) Вспомогательные функции для получения списка строк, по которым будем получать эмбеддинги

In [5]:
'''Батч будет состоять из набора всех строк, для одного рецепта, для которых мы хотим получить эмбеддинги
1) title - название рецепта
2) ingredients - ингредиенты
3) title_ingredients - название рецепта + ингредиенты
4) steps - для каждого шага отдельный эмбеддинг
5) all_steps - все шаги в виде одного текста
6) title_steps - название и все шаги в виде текста
7) all - название, ингредиенты и все шаги в виде одного текста
'''

def remove_period_if_needed(text: str) -> str:
    return text if text[-1] != '.' else text[:-1]

def add_period_if_needed(text: str) -> str:
    return text if text[-1] == '.' else text + '.'

def get_short_recipe_texts(recipe) -> List[str]:
    ''' По рецепту (строке из датасета) возвращает список строк
        - названия рецепта
        - эмбеддинги ингредиентов
        - отдельная строка для каждого шага
    '''
    texts = []
    title = recipe['title']
    # не хорошо будет, если название рецепта будет заканчиваться на точку
    title = remove_period_if_needed(title)
    texts.append(title)
    
    # ингредиенты будут списком через запятую
    ingredients = ', '.join(recipe['ingredients'].split('|'))
    ingredients = remove_period_if_needed(ingredients)
    texts.append(ingredients)
    
    # название рецепта + ингредиенты
    title_ingredients = '. '.join([title, ingredients])
    texts.append(title_ingredients)
    
    # шаги отдельно
    raw_steps = recipe['steps'].split('|')
    
    # Попадаются рецепты, где первый шаг пустой - добавил нейтральную фразу
    empty_steps = 0
    if raw_steps[0] == "":
        raw_steps[0] = "Подготовьте все ингредиенты"
        empty_steps += 1
        
    # Если есть пропуски внутри шагов - считаем, что 2 картинки относятся к предыдущему шагу
    for i in range(1, len(raw_steps)):
        if raw_steps[i] == "":
            empty_steps += 1
            raw_steps[i] = raw_steps[i - 1]
    steps = list(map(remove_period_if_needed, raw_steps))
    texts.extend(steps)
    
    return texts, empty_steps, len(raw_steps)


def get_long_recipe_texts(recipe) -> List[str]:
    ''' По рецепту (строке из датасета) возвращает список строк
        - все шаги одной строкой
        - название + все шаги одной строкой
        - название + ингредиенты + все шаги одной строкой
    '''
    texts = []
    
    title = recipe['title']
    # не хорошо будет, если название рецепта будет заканчиваться на точку
    title = remove_period_if_needed(title)
    
    # ингредиенты будут списком через запятую
    ingredients = ', '.join(recipe['ingredients'].split('|'))
    ingredients = remove_period_if_needed(ingredients)
    
    # шаги отдельно
    raw_steps = recipe['steps'].split('|')
    
    # Попадаются рецепты, где первый шаг пустой - добавил нейтральную фразу
    if raw_steps[0] == "":
        raw_steps[0] = "Подготовьте все ингредиенты"
        
    # Если есть пропуски внутри шагов - считаем, что 2 картинки относятся к предыдущему шагу
    for i in range(1, len(raw_steps)):
        if raw_steps[i] == "":
            raw_steps[i] = raw_steps[i - 1]
    steps = list(map(remove_period_if_needed, raw_steps))
    
    # шаги вместе
    all_steps = '. '.join(steps)
    texts.append(all_steps)

    # название и все шаги в виде текста
    title_all_steps = '. '.join([title, all_steps])
    texts.append(title_all_steps)

    # название, ингредиенты и все шаги в виде одного текста
    whole_recipe = '. '.join([title, ingredients, all_steps])
    texts.append(whole_recipe)
    return texts


### 4) Функции для получения эмбеддингов

In [6]:
def get_embeddings_for_short_texts():   
    ''' Функция возвращает 3 словаря, где ключи - id рецептов
    max_embeddings и mean_embeddings - словари с max и mean embedding'ами:
        каждый элемент - словарь:
            'title': эмбеддинги названия рецепта (768,)
            'ingredients': эмбеддинги ингредиентов (768,)
            'title_ingredients': эмбеддинги строки название_ингредиенты (768,)
            'steps': эмбеддинги каждого шага (n_steps, 768)
    missing_steps - значения - пары (empty_steps, n_steps)
    '''
    max_embeddings = {} # Здесь будем собираеть max эмбеддинги
    mean_embeddings = {} # Здесь будем собираеть mean эмбеддинги
    missing_steps = {}
    for i in tqdm.tqdm(range(len(data))):
        recipe_max_embs = {}
        recipe_mean_embs = {}
        recipe = data.iloc[i]
        # Embeddings для названия рецепта
        texts, empty_steps, total_steps = get_short_recipe_texts(recipe)
        missing_steps[recipe['id']] = (empty_steps, total_steps)
        try:
            _, _, _, _, sent_max_embs, sent_mean_embs, bert_pooler_outputs = m(texts)
        except RuntimeError as e:
            print(f'{i}: {e}')
            continue
        recipe_max_embs['title'] = sent_max_embs[0]
        recipe_mean_embs['title'] = sent_mean_embs[0]

        recipe_max_embs['ingredients'] = sent_max_embs[1]
        recipe_mean_embs['ingredients'] = sent_mean_embs[1]

        recipe_max_embs['title_ingredients'] = sent_max_embs[2]
        recipe_mean_embs['title_ingredients'] = sent_mean_embs[2]

        recipe_max_embs['steps'] = sent_max_embs[3:]
        recipe_mean_embs['steps'] = sent_mean_embs[3:]

        max_embeddings[recipe['id']] = recipe_max_embs
        mean_embeddings[recipe['id']] = recipe_mean_embs
    return max_embeddings, mean_embeddings, missing_steps

In [9]:
def get_embeddings_for_long_texts():
    ''' Функция возвращает 2 словаря, где ключи - id рецептов
    max_embeddings и mean_embeddings - словари с max и mean embedding'ами:
        каждый элемент - словарь:
            'all_steps': эмбеддинги строки со всеми шагами (768,)
            'title_all_steps': эмбеддинги строки название_все_шаги (768,)
            'whole_recipe': эмбеддинги строки весь_рецепт (768,)
    '''
    max_embeddings = {} # Здесь будем собираеть max эмбеддинги
    mean_embeddings = {} # Здесь будем собираеть mean эмбеддинги
    for i in tqdm.tqdm(range(len(data))):
        recipe_max_embs = {}
        recipe_mean_embs = {}
        recipe = data.iloc[i]
        # Embeddings для названия рецепта
        texts = get_long_recipe_texts(recipe)
        
        try:
            _, _, _, _, sent_max_embs, sent_mean_embs, bert_pooler_outputs = m(texts)
        except RuntimeError as e:
            continue

        recipe_max_embs['all_steps'] = sent_max_embs[0]
        recipe_mean_embs['all_steps'] = sent_mean_embs[0]

        recipe_max_embs['title_all_steps'] = sent_max_embs[1]
        recipe_mean_embs['title_all_steps'] = sent_mean_embs[1]
        
        recipe_max_embs['whole_recipe'] = sent_max_embs[2]
        recipe_mean_embs['whole_recipe'] = sent_mean_embs[2]

        max_embeddings[recipe['id']] = recipe_max_embs
        mean_embeddings[recipe['id']] = recipe_mean_embs
    return max_embeddings, mean_embeddings

### 5) Загружаем и сохраняем эмбеддинги

In [8]:
max_short_embeddings, mean_short_embeddings, missing_steps = get_embeddings_for_short_texts()

100%|██████████| 13830/13830 [10:52<00:00, 21.21it/s]


In [9]:
with open('RuBERT_max_embeddings_short.pkl', 'wb') as f:
    pickle.dump(max_short_embeddings, f)
    
with open('RuBERT_mean_embeddings_short.pkl', 'wb') as f:
    pickle.dump(mean_short_embeddings, f)
    
with open('missing_steps.pkl', 'wb') as f:
    pickle.dump(missing_steps, f)

In [10]:
max_long_embeddings, mean_long_embeddings = get_embeddings_for_long_texts()

100%|██████████| 13830/13830 [11:49<00:00, 19.48it/s]


In [11]:
with open('RuBERT_max_embeddings_long.pkl', 'wb') as f:
    pickle.dump(max_long_embeddings, f)
    
with open('RuBERT_mean_embeddings_long.pkl', 'wb') as f:
    pickle.dump(mean_long_embeddings, f)

In [13]:
print(f'Получили эмбеддингов всего (short): {len(max_short_embeddings)}')
print(f'Получили эмбеддингов полных рецептов (long): {len(max_long_embeddings)}')
print(f'Процент длинных рецептов: {(1 - len(max_long_embeddings)/len(max_short_embeddings))*100}')

Получили эмбеддингов всего (short): 13830
Получили эмбеддингов полных рецептов (long): 12667
Процент длинных рецептов: 8.409255242227044


In [23]:
with open('missing_steps.pkl', 'rb') as f:
    tmp = pickle.load(f)

### 6) Посмотрим сколько рецептов имеют пропуски

In [27]:
any_missing = 0
for k, v in missing_steps.items():
    if v[0] > 0:
        any_missing += 1

In [28]:
any_missing

255